# TDT4136 - Assignment 3
## Constraint Satisfaction Problems
### 1 - Import our implementation

In [ ]:
import Assignment as a

### 2 - Test our implementation on map colouring problem

In [ ]:
coloring_map = a.create_map_coloring_csp()
result = coloring_map.backtracking_search()
print(result)

### 3 - Solve different sudoku boards

In [ ]:
from pathlib import Path

sudokus = Path('').glob('*.txt')

for sudoku_file in sudokus:
    print(sudoku_file)
    sudoku = a.create_sudoku_csp(sudoku_file)
    solution = sudoku.backtracking_search()
    a.print_sudoku_solution(solution)

    print(f"\nNumber of backtrack function calls: {sudoku.number_of_calls}")
    print(f"Number of backtrack function failures: {sudoku.number_of_fails}")
    print('---------------------\n')

### 4 - Look at the arc-consistency at the beginning and possibilities

In [ ]:
import copy
import numpy as np

sudokus_ordered_by_difficulty = ['easy.txt', 'medium.txt', 'hard.txt', 'veryhard.txt']

y = []
possibilities_table_ = {}
for board in sudokus_ordered_by_difficulty:
        not_solved_sudoku = a.create_sudoku_csp(board)
        assignment = copy.deepcopy(not_solved_sudoku.domains)
        
        not_solved_sudoku.inference(assignment, not_solved_sudoku.get_all_arcs())

        possibilities_table_[board] = [ [len(assignment[f"{i}-{j}"]) for i in range(9)] for j in range(9)]
        y.append(np.prod(np.array(possibilities_table_[board])))

        print('board:', board)
        print('total possibilities:', np.prod(np.array(possibilities_table_[board])))
        print('mean possibilities:', np.mean(np.array(possibilities_table_[board])), end='\n\n')



In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(14, 5))
plt.subplot(1,2,1)
plt.plot(sudokus_ordered_by_difficulty, y)
plt.subplot(1,2,2)
plt.plot(sudokus_ordered_by_difficulty, np.log10(y))
plt.show()

### 5 - Look at the boards complexity

In [ ]:
sudokus_ordered_by_difficulty = ['easy.txt', 'medium.txt', 'hard.txt', 'veryhard.txt']

for board_file in sudokus_ordered_by_difficulty:
    print('Board difficulty:', board_file)
    board = list(map(lambda x: x.strip(), open(board_file, 'r')))
    k = 0
    for row in board: 
        for col in row: 
            if col == '0': k += 1
    print('Number of empty cells:', k, '\n')

In [ ]:
sudokus = Path('').glob('*.txt')

number_of_keys = {}

constraints_possible_lengths = {}

number_of_constraints_with_length_ = {}
current_constraints_with_length_ = {}
last_number_of_constraints_with_length_ = {}
is_always_same_number_of_constraints_for_length_ = {}
total_number_of_constraints = {}

for board in sudokus:
    print(board, end='\n\n')
    board_difficulty = str(board)
    number_of_keys[board_difficulty] = 0

    constraints_possible_lengths[board_difficulty] = []

    number_of_constraints_with_length_[board_difficulty] = {}
    current_constraints_with_length_[board_difficulty] = {}
    last_number_of_constraints_with_length_[board_difficulty] = {}
    is_always_same_number_of_constraints_for_length_[board_difficulty] = {}
    
    not_solved_sudoku = a.create_sudoku_csp(board)
    for key, value in not_solved_sudoku.constraints.items():
        number_of_keys[board_difficulty] += 1

        for constraint_key, constraint_value in value.items():
            if len(constraint_value) in constraints_possible_lengths[board_difficulty]:
                current_constraints_with_length_[board_difficulty][len(constraint_value)] += 1
            else: 
                constraints_possible_lengths[board_difficulty].append(len(constraint_value))
                current_constraints_with_length_[board_difficulty][len(constraint_value)] = 1
                number_of_constraints_with_length_[board_difficulty][len(constraint_value)] = 0
                last_number_of_constraints_with_length_[board_difficulty][len(constraint_value)] = 0
                is_always_same_number_of_constraints_for_length_[board_difficulty][len(constraint_value)] = True

        for constraint_length in constraints_possible_lengths[board_difficulty]:
            number_of_constraints_with_length_[board_difficulty][constraint_length] += current_constraints_with_length_[board_difficulty][constraint_length]
            is_always_same_number_of_constraints_for_length_[board_difficulty][constraint_length] = is_always_same_number_of_constraints_for_length_[board_difficulty][constraint_length] * (last_number_of_constraints_with_length_[board_difficulty][constraint_length] == current_constraints_with_length_[board_difficulty][constraint_length]) if last_number_of_constraints_with_length_[board_difficulty][constraint_length] != 0 else True

            last_number_of_constraints_with_length_[board_difficulty][constraint_length] = current_constraints_with_length_[board_difficulty][constraint_length]
            current_constraints_with_length_[board_difficulty][constraint_length] = 0


    print('number of keys:', number_of_keys[board_difficulty], end='\n\n')

    total_number_of_constraints[board_difficulty] = 0
    constraints_possible_lengths[board_difficulty].sort()
    for constraint_length in constraints_possible_lengths[board_difficulty]:
        print('Constraint length:', constraint_length)
        print(f'Number of constraints with a length of {constraint_length}:', number_of_constraints_with_length_[board_difficulty][constraint_length])

        print(f'Always same length for constraint with a length of {constraint_length}:', bool(is_always_same_number_of_constraints_for_length_[board_difficulty][constraint_length]))
        print('Size is always:', last_number_of_constraints_with_length_[board_difficulty][constraint_length]) if bool(is_always_same_number_of_constraints_for_length_[board_difficulty][constraint_length]) else print('Last length was:', last_number_of_constraints_with_length_[board_difficulty][constraint_length])
        total_number_of_constraints[board_difficulty] += number_of_constraints_with_length_[board_difficulty][constraint_length]

        print('\n')
    
    print('Total number of constraints for this board:', total_number_of_constraints[board_difficulty])
    print('\n---------------------\n')

In [ ]:
import matplotlib.pyplot as plt

sudokus_ordered_by_difficulty = ['easy.txt', 'medium.txt', 'hard.txt', 'veryhard.txt']

colors = ['red', 'blue', 'green']

constraint_lengths = []
for difficulty in sudokus_ordered_by_difficulty:
    for constraints_possible_length in constraints_possible_lengths[difficulty]:
        if not any(constraints_possible_length in tup for tup in constraint_lengths):
            constraint_lengths.append((constraints_possible_length, colors[0]))
            colors.pop(0)

y = { elem: [] for elem, _ in constraint_lengths }
total_constraints_values_in_ = { difficulty: 0 for difficulty in sudokus_ordered_by_difficulty }
plt.figure(figsize=(7, 5))

for elem, color in constraint_lengths:
    for difficulty in sudokus_ordered_by_difficulty:
        y[elem].append(number_of_constraints_with_length_[difficulty][elem])
        total_constraints_values_in_[difficulty] += elem * number_of_constraints_with_length_[difficulty][elem]
    plt.plot(sudokus_ordered_by_difficulty, y[elem], label=f"size {elem}", color=color)

plt.legend(loc='center right', title='Legend')
plt.show()

In [ ]:
total_constraints_values_in_

In [ ]:
e_ = { b: [[ 0 for _ in range(9)] for _ in range(9)] for b in sudokus_ordered_by_difficulty}

for board in sudokus_ordered_by_difficulty:
    not_solved_sudoku = a.create_sudoku_csp(board)

    for key, value in not_solved_sudoku.constraints.items():
        ik, jk = int(key[0]), int(key[2])
        for constraint_key, constraint_values in value.items():
            i, j = int(constraint_key[0]), int(constraint_key[2])

            if len(constraint_values) == 72 and i != ik and j != jk:
                e_[board][ik][jk] += 1

y = []
for board in sudokus_ordered_by_difficulty:
    print('board:', board)
    print('eijk:', np.sum(np.array(e_[board])), end='\n\n')
    y.append(np.sum(np.array(e_[board])))

plt.figure(figsize=(7,5))
plt.plot(sudokus_ordered_by_difficulty, y)
